# Day 1-00.03: RAFT 전처리 빠르게 따라가기

이 노트북에서는 Hugging Face의 한국어 RAG 데이터셋을 불러와 **RAFT(Retrieval-Augmented Fine-Tuning)** 템플릿 형태로 변환합니다.

## 오늘 흐름
1. 데이터셋 다운로드
2. RAFT 템플릿 샘플 생성
3. 통계 확인 및 저장 (로컬 / Google Drive)


### 1. 필요한 라이브러리 불러오기

In [ ]:
import json
import random
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset
from tqdm.auto import tqdm

print("✅ 라이브러리 로드 완료")


### 2. Hugging Face에서 데이터셋 가져오기

In [ ]:
DATASET_NAME = "neural-bridge/rag-dataset-12000"
print(f"📥 {DATASET_NAME} 데이터셋 다운로드 중...")

dataset = load_dataset(DATASET_NAME, split="train")
print(f"✅ 총 샘플 수: {len(dataset):,}")
print(f"✅ 포함된 필드: {dataset.column_names}")

### RAFT 템플릿이란?
**Retrieval-Augmented Fine-Tuning**은 질문과 여러 문서를 함께 제시해 모델이 정답 문서를 스스로 구분하도록 훈련하는 방식입니다.

- **Positive 문서**: 실제 정답이 담긴 문서 (모델이 참고해야 하는 문서)
- **Distractor 문서**: 일부러 포함한 오답 문서 (모델이 무시해야 하는 문서)
- **입력 포맷**: 질문 + 정답 문서 + 오답 문서를 하나의 텍스트로 묶고, 모델에게 자연스러운 답변을 생성하라고 요청합니다.

이렇게 학습하면 모델은 “어떤 정보를 믿어야 하는지”까지 파악하게 되며, 검색 기반 응용에 적합해집니다.


### 3. RAFT 템플릿으로 변환하는 함수 정의

In [ ]:
def create_raft_training_sample(question, answer, context, all_contexts, num_distractors=2):
    """주어진 질문/답변/정답 문서와 전체 문서를 이용해 RAFT 샘플을 생성합니다."""
    distractors = []
    while len(distractors) < num_distractors:
        candidate = random.choice(all_contexts)
        if candidate != context and candidate not in distractors:
            distractors.append(candidate)

    text = "".join([
        "질문: " + question,
        "정답 문서: " + context,
        "오답 문서: " + " || ".join(distractors),
        "위 정보를 참고하여 자연스러운 답변을 작성하세요.",
    ])

    return {
        "text": text,
        "question": question,
        "answer": answer,
        "context": context,
        "distractors": distractors,
    }


In [ ]:
def find_column(column_names, candidates, description):
    for name in candidates:
        if name in column_names:
            return name
    raise KeyError(f"필드 {description} 후보 {candidates} 중 하나를 찾지 못했습니다.")


def to_string(value):
    if isinstance(value, dict):
        for key in ["text", "passage", "content", "answer", "response"]:
            if key in value:
                return str(value[key])
    if isinstance(value, (list, tuple)):
        return str(value[0]) if value else ""
    return str(value)


def to_raft_dataset(dataset, num_distractors=2):
    print("🔄 RAFT 템플릿 변환 중...")
    column_names = dataset.column_names

    question_col = find_column(column_names, ["question", "query", "prompt"], "question")
    answer_col = find_column(column_names, ["answer", "positive_answer", "response"], "answer")
    context_col = find_column(
        column_names,
        ["positive_ctx", "positive_context", "context", "document", "passage", "answer_text"],
        "context",
    )

    all_contexts = [to_string(item[context_col]) for item in dataset]
    raft_records = []

    for item in tqdm(dataset, total=len(dataset)):
        question = to_string(item[question_col])
        answer = to_string(item[answer_col])
        context = to_string(item[context_col])

        raft_records.append(
            create_raft_training_sample(
                question=question,
                answer=answer,
                context=context,
                all_contexts=all_contexts,
                num_distractors=num_distractors,
            )
        )

    print("✅ RAFT 형식 변환 완료")
    return raft_records


### 4. 실제로 변환을 실행하고 샘플 확인

In [ ]:
raft_records = to_raft_dataset(dataset, num_distractors=2)
print(f"총 RAFT 샘플 수: {len(raft_records):,}")

for idx in range(2):
    sample = raft_records[idx]
    print("━" * 40)
    print(f"샘플 {idx + 1}")
    print(f"질문: {sample['question'][:80]}...")
    print(f"정답: {sample['answer'][:80]}...")
    print(f"Distractors 개수: {len(sample['distractors'])}")

### 5. 텍스트 길이 등 간단한 통계 요약

In [ ]:
text_lengths = [len(record["text"]) for record in raft_records]
question_lengths = [len(record["question"]) for record in raft_records]
answer_lengths = [len(record["answer"]) for record in raft_records]

print("📊 전처리 결과 요약")
print(f"  • 평균 텍스트 길이: {np.mean(text_lengths):.1f}")
print(f"  • 평균 질문 길이: {np.mean(question_lengths):.1f}")
print(f"  • 평균 답변 길이: {np.mean(answer_lengths):.1f}")
print(f"  • distractor 문서 수 (per sample): 2")

### 6. Hugging Face Dataset 형태로 변환 후 저장하기

In [ ]:
raft_dataset = Dataset.from_list(raft_records)

SAVE_DIR = "data/raft_dataset"
raft_dataset.save_to_disk(SAVE_DIR)

stats = {
    "total_samples": len(raft_records),
    "avg_text_length": float(np.mean(text_lengths)),
    "avg_question_length": float(np.mean(question_lengths)),
    "avg_answer_length": float(np.mean(answer_lengths)),
    "num_distractors": 2,
}

with open("data/raft_dataset_stats.json", "w", encoding="utf-8") as f:
    json.dump(stats, f, ensure_ascii=False, indent=2)

print("💾 저장 완료: data/raft_dataset/ 및 data/raft_dataset_stats.json")


### 7. (선택) Google Drive에도 저장해 두기
Colab에서 실행한다면 아래 셀을 실행해 Drive에 동일한 폴더를 복사할 수 있습니다.

In [ ]:
GOOGLE_DRIVE_DIR = "/content/drive/MyDrive/exaone_day1/data/raft_dataset"

try:
    from google.colab import drive  # type: ignore
    import shutil
    drive.mount('/content/drive')
    shutil.copytree("data/raft_dataset", GOOGLE_DRIVE_DIR, dirs_exist_ok=True)
    shutil.copy("data/raft_dataset_stats.json", GOOGLE_DRIVE_DIR + "_stats.json")
    print(f"✅ Google Drive에 저장 완료: {GOOGLE_DRIVE_DIR}")
except ModuleNotFoundError:
    print("⚠️ Colab 환경이 아닙니다. 필요하면 직접 경로를 수정해 저장하세요.")
except NotImplementedError:
    print("⚠️ 이 환경에서는 drive.mount()를 사용할 수 없습니다. Colab 노트북에서 실행해주세요.")


In [ ]:
# from datasets import Dataset

# # Arrow 파일 직접 로드
# dataset = Dataset.from_file("data-00000-of-00001.arrow")

# # 샘플 보기
# print(dataset)
# print(dataset[0])   # 첫 번째 row 출력


### 8. 다음 단계
- 저장된 `data/raft_dataset`을 main-practice/03 노트북에서 불러와 파인튜닝에 사용합니다.
- Drive에도 복사해 두었다면 다른 런타임에서도 즉시 접근할 수 있습니다.